# 🖊️ SignVerifAI - Signature Verification with Siamese Networks

Bu notebook, imza doğrulama modelini **Google Colab GPU** üzerinde eğitmek için hazırlanmıştır.

**Gereksinimler:**
- Google Drive'a yüklenmiş veri seti
- Colab GPU runtime (T4 veya A100)

---

## 1️⃣ GPU Kontrolü

In [ ]:
# GPU kontrolü
!nvidia-smi

import torch
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")

## 2️⃣ Repo'yu Klonla

In [ ]:
# GitHub'dan klonla
!git clone https://github.com/gorkemelih/SignVerifAI.git
%cd SignVerifAI

In [ ]:
# Bağımlılıkları kur
!pip install -e . -q
print("✅ Kurulum tamamlandı!")

## 3️⃣ Google Drive'ı Bağla

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Drive'dan verileri kopyala
# NOT: Önce Drive'a data_processed.zip yüklemelisin!

DRIVE_DATA_PATH = "/content/drive/MyDrive/SignVerifAI/data_processed.zip"

import os
if os.path.exists(DRIVE_DATA_PATH):
    !unzip -q {DRIVE_DATA_PATH} -d .
    print("✅ Veriler yüklendi!")
else:
    print(f"❌ Dosya bulunamadı: {DRIVE_DATA_PATH}")
    print("👉 Lütfen Drive'a data_processed.zip yükleyin.")

## 4️⃣ Veri Hazırlığı

In [ ]:
# Metadata kontrolü
import pandas as pd

metadata = pd.read_csv('data_processed/metadata.csv')
print(f"Toplam görüntü: {len(metadata)}")
print(f"Kişi sayısı: {metadata['person_id'].nunique()}")
print(f"\nEtiket dağılımı:")
print(metadata['label'].value_counts())

In [ ]:
# Split oluştur
!signverify split

# Pair'leri oluştur (50k train, 10k val)
!signverify pairs

## 5️⃣ Model Eğitimi 🚀

In [ ]:
# Eğitim parametreleri
EPOCHS = 50
BATCH_SIZE = 64  # GPU için daha büyük batch
LEARNING_RATE = 1e-4

# Eğitimi başlat
!signverify train --device cuda --epochs {EPOCHS} --batch-size {BATCH_SIZE} --lr {LEARNING_RATE}

## 6️⃣ Değerlendirme

In [ ]:
# Test seti üzerinde değerlendirme
!signverify eval --device cuda

In [ ]:
# ROC eğrisi görüntüle
from IPython.display import Image
Image('outputs/reports/roc_curve.png')

In [ ]:
# Score dağılımı görüntüle
Image('outputs/reports/score_distribution.png')

## 7️⃣ Modeli Kaydet (Drive'a)

In [ ]:
# En son modeli Drive'a kaydet
import shutil
import glob

# En son run klasörünü bul
runs = sorted(glob.glob('outputs/models/run_*'))
if runs:
    latest_run = runs[-1]
    dest = "/content/drive/MyDrive/SignVerifAI/trained_models/"
    os.makedirs(dest, exist_ok=True)
    shutil.copytree(latest_run, dest + os.path.basename(latest_run))
    print(f"✅ Model kaydedildi: {dest}")
else:
    print("❌ Eğitilmiş model bulunamadı.")

---
## 📊 Sonuçlar

Eğitim tamamlandıktan sonra:
- `outputs/reports/eval_report.md` - Detaylı değerlendirme raporu
- `outputs/reports/roc_curve.png` - ROC eğrisi
- `outputs/models/run_*/checkpoint_best.pt` - En iyi model